<a href="https://colab.research.google.com/github/adithyaprabhu007/math-coding-notes/blob/main/multiple_logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Generate IQ and CGPA
n = 100
iq = np.random.normal(100, 10, n)        # Normally distributed IQ around 100
cgpa = np.random.uniform(5.0, 10.0, n)   # Uniform CGPA between 5 and 10

# Define linearly separable logic
# We'll define a linear score and classify based on thresholds
score = 0.04 * (iq - 100) + 0.6 * (cgpa - 7)

# Assign categories
result = np.where(score > 1.0, 'distinction',
         np.where(score > 0.0, 'pass', 'fail'))

# Add 2% label noise
num_noisy = int(0.02 * n)
noise_indices = np.random.choice(n, num_noisy, replace=False)

for i in noise_indices:
    current = result[i]
    other_labels = ['pass', 'fail', 'distinction']
    other_labels.remove(current)
    result[i] = np.random.choice(other_labels)

# Create DataFrame
df = pd.DataFrame({
    'iq': iq,
    'cgpa': cgpa,
    'result': result
})

print(df.head())


           iq      cgpa       result
0  104.967142  7.087055         pass
1   98.617357  6.110539         fail
2  106.476885  5.599327         fail
3  115.230299  6.688076         pass
4   97.658466  9.714549  distinction


In [8]:
df.describe()


,iq,cgpa
count,100.000000,100.000000
mean,98.961535,7.431031
std,9.081684,1.440720
min,73.802549,5.025308
25%,93.990943,6.206398
50%,98.730437,7.536930
75%,104.059521,8.473381
max,118.522782,9.928252


In [9]:
df.sample()


,iq,cgpa,result
17,103.142473,7.513395,pass


In [10]:
import pandas as pd
import numpy as np

# --- Preprocessing ---
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Encode the labels
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(df['result'])  # 0,1,2 for fail, pass, distinction

# One-hot encode the targets
y_onehot = np.zeros((len(y_encoded), 3))
y_onehot[np.arange(len(y_encoded)), y_encoded] = 1

# Feature matrix
X = df[['iq', 'cgpa']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add bias column (intercept)
X_scaled = np.hstack([X_scaled, np.ones((X_scaled.shape[0], 1))])  # shape: (100, 3)

# --- Softmax Function ---
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# --- Cross Entropy Loss ---
def cross_entropy(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))  # +1e-9 to avoid log(0)

# --- Gradient Descent ---
def train(X, y, lr=0.1, epochs=1000):
    n_samples, n_features = X.shape
    n_classes = y.shape[1]

    # Initialize weights randomly
    W = np.zeros((n_features, n_classes))  # shape: (3, 3)

    for epoch in range(epochs):
        logits = np.dot(X, W)               # shape: (100, 3)
        y_pred = softmax(logits)            # shape: (100, 3)
        loss = cross_entropy(y, y_pred)

        # Gradient of loss w.r.t weights
        grad = np.dot(X.T, (y_pred - y)) / n_samples  # shape: (3, 3)

        W -= lr * grad

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {loss:.4f}")

    return W

# --- Train the model ---
W_trained = train(X_scaled, y_onehot, lr=0.1, epochs=1000)

# --- Predictions ---
def predict(X, W):
    logits = np.dot(X, W)
    probs = softmax(logits)
    return np.argmax(probs, axis=1)

y_pred = predict(X_scaled, W_trained)
acc = np.mean(y_pred == y_encoded)
print(f"\nAccuracy: {acc:.2%}")


Epoch 0: Loss = 1.0986
Epoch 100: Loss = 0.5642
Epoch 200: Loss = 0.5016
Epoch 300: Loss = 0.4738
Epoch 400: Loss = 0.4584
Epoch 500: Loss = 0.4488
Epoch 600: Loss = 0.4424
Epoch 700: Loss = 0.4379
Epoch 800: Loss = 0.4347
Epoch 900: Loss = 0.4324

Accuracy: 94.00%
